This script takes Texas Education Agency data about school district demographics and disciplinary actions, and puts them together in one GeoJSON file for the Texas Appleseed "School to Prison Pipeline" map. See http://www.texasdisciplinelab.org/. A newer and hopefully easier way of processing the TEA's data is to use the command line utility in the /makedata/ directory. However, the output format is different from the format that would be produced by this notebook. Check txappleseedmap's README.MD for details.

To use the script, follow these instructions:

1. For every year that you want to cover, download all 20 of the region files from http://rptsvr1.tea.texas.gov/adhocrpt/Disciplinary_Data_Products/Download_Region_Districts.html and put them in the directory '../data/from_agency/by_region/'

2. For every year that you want to cover, download the "District and Charter Detail Data" Snapshot Data File (comma-delimited *.dat)" from https://rptsvr1.tea.texas.gov/perfreport/snapshot/download.html. The website automatically delivers these files with the same filename: district.dat. You will need to rename them to have different names by adding the year after "district". For instance, "district2016.dat"

3. This script needs a GeoJSON file of district shapes. Make sure it can find that file at '../geojson/base_districts.geojson'

4. Change the first_year and last_year variables below to reflect the years you want your file to cover.

5. Run the notebook with "Kernel -> Restart and Run All"

6. Wait a while for it to finish. After about 15 minutes, the notebook should produce 'districts_with_data.geojson' in the '../geojson/' directory.

7. The resulting file will be about 20 MB depending on how many years it covers. You can make it smaller (about 10 MB) by uploading it to http://mapshaper.org/, using the "simplify" function to reduce the number of lines in the district boundaries, and exporting the file as TopoJSON instead of GeoJSON. I did this and put the result in the '../topojson/' directory.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

pd.options.display.max_columns = 999

first_year = 2006 # the year 2006 is the first year on the TEA site
last_year = 2016


def formatDF(apple, year_col):
    
    # Removes rows and columns not needed for the map
    
    apple = apple.drop(["AGGREGATION LEVEL","REGION","DISTNAME"], axis = 1)
    
    # Adding totals for each discipline in each district, by adding up actions 
    # against special ed students and non-special ed students. This will be 
    # inefficient because it makes a dict list first instead of staying in pandas.
    
    non_special = {"D06": ("D05","D-EXPULSION ACTIONS"), 
                   "D09": ("D08","E-DAEP PLACEMENTS"), 
                   "D12": ("D11", "F-OUT OF SCHOOL SUSPENSIONS"), 
                   "D15": ("D14", "G-IN SCHOOL SUSPENSIONS")}
    
    all_actions = []
    
    # if it was a .csv, the headers would be ["DISTRICT", "SECTION", "HEADING", "HEADING NAME", year_col]
    
    unfound = []
    
    for d in apple.index.get_level_values(0).unique():
        for key in non_special:
            try: 
                a = apple.loc[(d, key)][year_col]
            except KeyError:
                a = 0
            try:
                b = apple.loc[(d, non_special[key][0])][year_col]
            except KeyError:
                b = 0
            if a < 0: # in case of dummy values like -999
                a = 1
            if b < 0:
                b = 1
            total = a + b
            all_actions.append({"DISTRICT": d, "HEADING": key, "SECTION": non_special[key][1], 
                                "HEADING NAME": "ALL", year_col: total})
    
    new = pd.DataFrame(all_actions)
    new = new.set_index(["DISTRICT", "HEADING"])

    # Keeping only the rows that categorize students by protected class.
    
    patternIn = 'WHITE|BLACK OR AFRICAN AMERICAN|AMERICAN INDIAN OR ALASKA NAT|HISPANIC|NATIVE HAWAIIAN|ASIAN|TWO OR MORE RACES|SPEC. ED|ECO. DISAD|ECO DISAD.|TOTAL'
    apple = apple[apple["HEADING NAME"].str.contains(patternIn)]
    
    # Getting rid of rows that count students instead of incidents, or non-disadvantaged kids.
    
    patternOut = 'SPEC. ED. STUDENTS| SPEC. ED. EXPULSIONS TO JJAEP|ECO DISAD. STUDENTS|ECO. DISAD. STUDENTS|AT RISK|NON AT|UNKNOWN AT|NON SPEC. ED.|NON ECO DISAD.|NON ECO. DISAD.'
    apple = apple[apple["HEADING NAME"].str.contains(patternOut) == False]

    # Delete rows appearing to double-count the same expulsions.
    
    JJAEPReplace = {"SECTION": {
                        'M-ECO\. DISADV\. JJAEP PLACEMENTS|H-SPEC\. ED\. JJAEP EXPULSIONS': 'C-JJAEP EXPULSIONS'}}
    apple = apple.replace(to_replace=JJAEPReplace, regex=True)
    apple = apple[apple["SECTION"].str.contains("JJAEP EXPULSIONS|DISCIPLINE ACTION COUNTS") == False]
                    
    apple = apple.append(new)
    
    # Consolidating some of the descriptors into broader categories
    
    appleReplace = {year_col:
                        {-99999999: 1, -999999: 1, -999: 1},
                    "SECTION": {
                        'D-EXPULSION ACTIONS|N-ECO\. DISADV\. EXPULSIONS|I-SPEC\. ED\. EXPULSIONS': 'EXP',
                        'E-DAEP PLACEMENTS|O-ECO\. DISADV\. DAEP PLACEMENTS|J-SPEC\. ED\. DAEP PLACEMENTS': 'DAE',
                        'F-OUT OF SCHOOL SUSPENSIONS|P-ECO\. DISADV\. OUT OF SCHOOL SUS.|K-SPEC\. ED\. OUT OF SCHOOL SUS\.': 'OSS',
                        'G-IN SCHOOL SUSPENSIONS|Q-ECO\. DISADV\. IN SCHOOL SUS\.|L-SPEC\. ED\. IN SCHOOL SUS\.': 'ISS'},
                    "HEADING NAME": {'SPEC\. ED.*$': 'SPE',
                                     'ECO?. DISAD.*$': 'ECO',
                                     'HISPANIC': 'HIS',
                                     'HIS/LATINO': 'HIS',
                                     'HISPANIC/LATINO': 'HIS',
                                     'BLACK OR AFRICAN AMERICAN': 'BLA',
                                     'BLACK/AFRICAN AMERICAN': 'BLA',
                                     'WHITE': 'WHI',
                                     'AMERICAN INDIAN OR ALASKA NAT': 'IND',
                                     'ASIAN': 'ASI',
                                     'NATIVE HAWAIIAN/OTHER PACIFIC': 'PCI',
                                     'TWO OR MORE RACES': 'TWO',
                                    }
                    }

    apple = apple.replace(to_replace=appleReplace, regex=True)
    
    return apple

def getYear(year):
    year_col = "YR{}".format(str(year)[-2:])
    apple_path = '../data/from_agency/by_region/REGION_{}_DISTRICT_summary_{}.csv'
    one_year = [pd.read_csv(apple_path.format(str(region).zfill(2),str(year)[-2:]), 
                            index_col = ["DISTRICT","HEADING"], dtype = {year_col: int})
                for region in range(1,21)]
    a = pd.concat(one_year)
    
    # a = a.set_index(["DISTRICT","HEADING"] )
    a = a[~a.index.duplicated(keep='last')]  # a single row was causing a non-unique multiindex error 
    # print(a.loc[31901])
    a = formatDF(a, year_col)
    return a

In [2]:
def populations(districtPath):
    district = pd.read_csv(districtPath, index_col="DISTRICT")

    district = district.rename(columns = {"SNAPDIST": 'DISTNAME'})
    
    sometimes_missing = [ 'DPETINDP', 'DPETASIP', 'DPETPCIP', 'DPETTWOP']
    
    for c in sometimes_missing:
        if c not in district.columns:
            district[c] = np.nan
    
    # deleting redundant columns
    
    district = district[['DISTNAME', 'REGION', 'DPETALLC', 
                         'DPETBLAP', 'DPETHISP', 'DPETWHIP', 'DPETINDP',
                         'DPETASIP', 'DPETPCIP', 'DPETTWOP', 'DPETECOP', 
                         'DPETSPEP']] # 'DISTRICT' not listed because it's the index

    return district




def getRatio(distPop, racePop, all_punishments, group_punishments):
    # Calculating ratio of punishments for the demographic group compared to the punishments for the student population
    # as a whole. For instance, "0.505" in the disparity column indicates the group got the punishment 50.5% as often
    # as average for the student population.

    """
    >>> getRatio(200, 20, 20, 10)
    4.0
    >>> getRatio(200, 20, 20, 2)
    0.0
    >>> print(getRatio(200, 0, 20, 0))
    None
    """

    if max(racePop, group_punishments) == 0 or None:
        return None
    elif all_punishments == 0 or None:
        return 0
    else:
        disparity = (group_punishments / (max(all_punishments, group_punishments))
                     / (max(racePop, group_punishments) / distPop)) - 1
        disparity = Decimal(disparity)
        disparity = disparity.quantize(Decimal('0.01'))
    return float(disparity)

In [3]:
def impossible(distPop, raceP, all_punishments, group_punishments):

    """
    >>> print(impossible(50, 20, 20, 100))
    1
    >>> impossible(20, 0, 20, 0)
    0
    """

    # The "RecordError" column flags implausible data entries. Some of them could still be true if school administrators
    # applied different standards different standards to determine which students belong to which demographic group.
    # Or some could be the result of students not being counted because of the time they moved in and out of district.

    if group_punishments > max(all_punishments,8): # eight because TEA could report 2 masked columns with 4 each
        return 1
    if raceP == 0 and group_punishments > 0:
        return 1
    return 0



def getFisher(distPop, racePop, all_punishments, group_punishments):

    """
    >>> getFisher(20, 5, 20, 10)
    2
    >>> getFisher(20, 0, 20, 0)
    None
    """
    
    # I don't know if this is a valid way to report the Fisher's exact test statistic, but the idea is that if getFisher returns a
    # positive number over .95, there's a 95% chance that the group's better-than-average treatment is not due to chance.
    # If it returns a number under -.95, there's a 95% chance that the group's worse-than-average treatment is not due to chance.
    # I think it should be easier to create a color scale to show the scores on a map this way.

    # The getFisher function assumes wrongly that everyone can have only one punishment (of each type). If the number of
    # punishments exceeds the number of kids, it reduces the number of punishments (and assumes wrongly that every
    # kid has been punished) But maybe the results are still close enough to correct to use for scaling?

    
    if max(racePop, group_punishments) == 0 or None:
        return None
    if distPop == 0:
        return None
    elif max(group_punishments, all_punishments) == 0 or None:
        return 0
    else:
        try: 
            oddsratio, pvalueG = stats.fisher_exact([[racePop, max(distPop - racePop, 0)],
                                                 [group_punishments, max(all_punishments - group_punishments, 0)]],
                                                alternative='greater')
        except ValueError:
            print(distPop, racePop, all_punishments, group_punishments)
        oddsratio, pvalueL = stats.fisher_exact([[racePop, max(distPop - racePop, 0)],
                                                 [group_punishments, max(all_punishments - group_punishments, 0)]],
                                                alternative='less')
        if pvalueL < pvalueG:
            pv = 1 - pvalueL
        else:
            pv = pvalueG - 1
        
        # To save space in the JSON, this simplifies the decimal values to an integer from -6 to 6
        # It should replace similar code in txappleseedmap/js/index.js
        
        scale = -6
        scale_colors = (-0.99999,-0.9984,-0.992,-0.96,-0.8,-0.2,0.2,0.8,0.96,0.992,0.9984,0.99999)
        
        for v in scale_colors:
            if pv > v:
                scale += 1
        
        # pv = Decimal(pv)
        # pv = pv.quantize(Decimal('0.000001'))
    return scale

print(getFisher(20, 5, 20, 10))

2


In [4]:
import json
with open("../geojson/base_districts.geojson") as json_data:
    district_map = json.load(json_data)
    json_data.close()

In [5]:
shapeIDs = set()

for shape in district_map["features"]:
    shape["id"] = shape["properties"]["DISTRICT_N"]
    assert shape["id"] not in shapeIDs, "id already in list: %r" % shape["id"]
    shapeIDs.add(shape["id"])
    
    # These two fields look redundant. Let's try deleting them.
    
    shape["properties"].pop("DISTRICT_1", None)
    shape["properties"].pop("OBJECTID_1", None)


In [6]:
type(district_map["features"][1]['geometry']['coordinates'][0][1][1])

float

In [7]:
# For districts overall, need columns that show what percentage of the state population they have
# and what percentage of the punishments?

def getLE(x):
    
    # Collects the correct values from the dataframes called "apple" and "district"
    # and calls the "impossible" function, which looks for data errors
    
    distPop = x["DPETALLC"]
    if distPop in (0, None, np.nan):
        return 1
    elif x["HEADING NAME"] == "ALL":
        return 0
    else:    
        all_punishments = x["all_punishments"]
        # all_punishments = apple.loc[x["DISTRICT"]][x["SECTION"]]["ALL"]
        group_punishments = x[year_col]
        # trying to make this run faster by returning info for two columns, then splitting them
        raceP = x["DPET{}P".format(x["HEADING NAME"][:3])]
        return impossible(distPop, raceP, all_punishments, group_punishments)


In [8]:
def getScale(x, punishment_totals, statewide_students_count):
    
    """
    This function does something different for the "HEADING NAME == ALL" rows than for the other rows.
    For the "ALL" rows it uses the whole state population as the "distPop" and uses the entire district population
    as the "racePop". For the other rows, the entire district population is used as "distPop", not "racePop".
    
    And this function calls getFisher for the real calculation.
    """
    
    group_punishments = x[year_col]
    if x["HEADING NAME"] == "ALL":
        distPop = statewide_students_count
        racePop = x["DPETALLC"]
        all_punishments = punishment_totals[x["SECTION"]]
    else:
        distPop = x["DPETALLC"]
        racePop = x["DPET{}P".format(x["HEADING NAME"])] * distPop * .01
        if pd.isna(racePop):
            return None
        if pd.isnull(x["all_punishments"]):
            print("null all_punishments: " + str(x))
        all_punishments = x["all_punishments"]
    return getFisher(distPop, racePop, all_punishments, group_punishments)

def getPercentage(x, punishment_totals):
    if x["HEADING NAME"] == "ALL":
        return x["all_punishments"] / punishment_totals[x["SECTION"]] * 100
    else:
        return x[year_col] / x["all_punishments"] * 100

In [9]:
# Need to merge columns of apple and district.

years = [x for x in range(first_year, last_year + 1)] # change back to first_year

pop_stats = ("DPETALLC","DPETALLP", "DPETBLAP","DPETHISP","DPETWHIP","DPETINDP","DPETASIP","DPETPCIP",
             "DPETTWOP","DPETECOP","DPETSPEP")

demos = ('ALL','SPE', 'ECO','HIS','BLA', 'WHI','IND', 'ASI','PCI', 'TWO')

punishments = ('EXP','DAE','OSS','ISS')

fail = {} # for testing
noScale = {}

for year in years:
    print("starting year " + str(year))

    apple = getYear(year)
    # the path to the files in the district demographics directory
    districtPath = '../data/from_agency/districts/district{}.dat'.format(year)
    district = populations(districtPath)

    statewide_students_count = district["DPETALLC"].sum()
    year_col = "YR" + str(year)[-2:]

    apple = apple.reset_index()
    appleAll = apple[apple["HEADING NAME"] == "ALL"].rename({year_col: "all_punishments"}, axis = 1).drop(["HEADING NAME", "HEADING"], axis = 1)
    print(appleAll[:5])
    apple = apple.merge(district, how = "left", left_on = "DISTRICT", right_index = True)
    
    apple = apple[apple["DPETALLC"].notnull()]
    
    punishment_totals = {}
    for p in apple["SECTION"].unique():
        punishment_totals[p] = apple[apple["SECTION"] == p][apple["HEADING NAME"] == "ALL"][year_col].sum()
        
    # apple[18464:18470]  previous problem rows, gone because of the .notnull()
    
    apple = apple.merge(appleAll, how = "left", left_on = ["DISTRICT","SECTION"], right_on = ["DISTRICT","SECTION"])
    
    """    
    # This line will run slowly because for each row, it searches the entire dataframe
    apple["all_punishments"] = apple.apply(lambda x: 
                                               apple[apple["DISTRICT"] == x["DISTRICT"]][apple["SECTION"] == x["SECTION"]][apple["HEADING NAME"] == "ALL"][year_col].values[0], axis=1)
    """
    
    # New column will show the district's percentage of the state's student population.
    district["DPETALLP"] = district.apply(lambda x: x["DPETALLC"] / statewide_students_count * 100, axis=1).round(2)
    
    apple["LikelyError"] = apple.apply(getLE, axis=1)
    apple["Scale"] = apple.apply(lambda x: getScale(x, punishment_totals, statewide_students_count), axis=1)
    apple["Percentage"] = apple.apply(lambda x: getPercentage(x, punishment_totals), axis=1).round(3)
    
    apple = apple.set_index(["DISTRICT","SECTION","HEADING NAME"])
    apple = apple.sort_index() # trying to improve speed
    
    # populating the GeoJSON file, which already has geometry for the districts.
    
    for entry in district_map["features"]:
        if entry["id"] in district.index:
            entry["properties"][year] = {}
            for stat in pop_stats:
                # This will give NaN (numpy.float64) when empty
                if pd.notnull(district.loc[entry["id"]][stat]):
                    try:
                        entry["properties"][year][stat] = district.loc[entry["id"]][stat]
                    except KeyError:
                        # for when the map has a district not in the TEA's data
                        print("no stats for " + str(year) + " " + str(entry["id"]))
                        entry["properties"][year][stat] = None
        if entry["id"] in apple.index.get_level_values(0):
            for punishment in punishments:
                entry["properties"][year][punishment] = {}
                for demo in demos:
                    if (entry["id"],punishment,demo) in apple.index:
                    # if pd.notnull(apple.loc[entry["id"],punishment,demo][year_col]): # should prevent empty dicts at "demo" level
                        entry["properties"][year][punishment][demo] = {} 
                        try:
                            entry["properties"][year][punishment][demo]["C"] = int(apple.loc[entry["id"],punishment,demo][year_col])
                            entry["properties"][year][punishment][demo]["E"] = int(apple.loc[entry["id"],punishment,demo]["LikelyError"])
                            entry["properties"][year][punishment][demo]["%"] = float(apple.loc[entry["id"],punishment,demo]["Percentage"])
                        except:
                            fail[entry["id"]] = (year,punishment,demo)
                        try:
                            entry["properties"][year][punishment][demo]["S"] = int(apple.loc[entry["id"],punishment,demo]["Scale"])
                        except:
                            noScale[entry["id"]] = (year,punishment,demo)
    print(district_map["features"][30]["properties"])
                    # print("Nothing for {} {} {}".format(entry["id"],punishment,demo))
                    # impossible(distPop, racePop, all_punishments, group_punishments)

starting year 2006
       DISTRICT SECTION  all_punishments
12970     31901     EXP               73
12971     31901     DAE              826
12972     31901     OSS             3144
12973     31901     ISS            15309
12974    108902     EXP               18


/Users/Matt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


{'DISTRICT_C': '228905', 'DISTRICT_N': 228905, 'OBJECTID': 31, 'NAME2': 'Apple Springs', 'DISTRICT': '228-905', 'OBJECTID_2': 1113, 'DISTNAME': 'APPLE SPRINGS ISD', 'REGION': 6, 2006: {'DPETALLC': 167, 'DPETALLP': 0.0, 'DPETBLAP': 16, 'DPETHISP': 2, 'DPETWHIP': 83, 'DPETECOP': 59.9, 'DPETSPEP': 20, 'EXP': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -1}}, 'DAE': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -4}}, 'OSS': {'ALL': {'C': 2, 'E': 0, '%': 0.0, 'S': -6}, 'SPE': {'C': 1, 'E': 0, '%': 50.0, 'S': 1}, 'ECO': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}, 'WHI': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}}, 'ISS': {'ALL': {'C': 30, 'E': 0, '%': 0.002, 'S': -5}, 'SPE': {'C': 29, 'E': 0, '%': 96.667, 'S': 6}, 'ECO': {'C': 20, 'E': 0, '%': 66.667, 'S': 1}, 'WHI': {'C': 25, 'E': 0, '%': 83.333, 'S': -1}}}}
starting year 2007
       DISTRICT SECTION  all_punishments
13039     31901     EXP              161
13040     31901     DAE             1083
13041     31901     OSS             3797
13042     31901     ISS  

       DISTRICT SECTION  all_punishments
13065     31901     EXP               58
13066     31901     DAE              889
13067     31901     OSS             4952
13068     31901     ISS            16379
13069    108902     EXP                2
{'DISTRICT_C': '228905', 'DISTRICT_N': 228905, 'OBJECTID': 31, 'NAME2': 'Apple Springs', 'DISTRICT': '228-905', 'OBJECTID_2': 1113, 'DISTNAME': 'APPLE SPRINGS ISD', 'REGION': 6, 2006: {'DPETALLC': 167, 'DPETALLP': 0.0, 'DPETBLAP': 16, 'DPETHISP': 2, 'DPETWHIP': 83, 'DPETECOP': 59.9, 'DPETSPEP': 20, 'EXP': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -1}}, 'DAE': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -4}}, 'OSS': {'ALL': {'C': 2, 'E': 0, '%': 0.0, 'S': -6}, 'SPE': {'C': 1, 'E': 0, '%': 50.0, 'S': 1}, 'ECO': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}, 'WHI': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}}, 'ISS': {'ALL': {'C': 30, 'E': 0, '%': 0.002, 'S': -5}, 'SPE': {'C': 29, 'E': 0, '%': 96.667, 'S': 6}, 'ECO': {'C': 20, 'E': 0, '%': 66.667, 'S': 1}, 'WHI': {'C':

       DISTRICT SECTION  all_punishments
17396     31901     EXP               51
17397     31901     DAE              848
17398     31901     OSS             3717
17399     31901     ISS            14344
17400    108902     EXP                2
{'DISTRICT_C': '228905', 'DISTRICT_N': 228905, 'OBJECTID': 31, 'NAME2': 'Apple Springs', 'DISTRICT': '228-905', 'OBJECTID_2': 1113, 'DISTNAME': 'APPLE SPRINGS ISD', 'REGION': 6, 2006: {'DPETALLC': 167, 'DPETALLP': 0.0, 'DPETBLAP': 16, 'DPETHISP': 2, 'DPETWHIP': 83, 'DPETECOP': 59.9, 'DPETSPEP': 20, 'EXP': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -1}}, 'DAE': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -4}}, 'OSS': {'ALL': {'C': 2, 'E': 0, '%': 0.0, 'S': -6}, 'SPE': {'C': 1, 'E': 0, '%': 50.0, 'S': 1}, 'ECO': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}, 'WHI': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}}, 'ISS': {'ALL': {'C': 30, 'E': 0, '%': 0.002, 'S': -5}, 'SPE': {'C': 29, 'E': 0, '%': 96.667, 'S': 6}, 'ECO': {'C': 20, 'E': 0, '%': 66.667, 'S': 1}, 'WHI': {'C':

       DISTRICT SECTION  all_punishments
17394     31901     EXP               38
17395     31901     DAE              671
17396     31901     OSS             2715
17397     31901     ISS             8848
17398    108902     EXP                2
{'DISTRICT_C': '228905', 'DISTRICT_N': 228905, 'OBJECTID': 31, 'NAME2': 'Apple Springs', 'DISTRICT': '228-905', 'OBJECTID_2': 1113, 'DISTNAME': 'APPLE SPRINGS ISD', 'REGION': 6, 2006: {'DPETALLC': 167, 'DPETALLP': 0.0, 'DPETBLAP': 16, 'DPETHISP': 2, 'DPETWHIP': 83, 'DPETECOP': 59.9, 'DPETSPEP': 20, 'EXP': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -1}}, 'DAE': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -4}}, 'OSS': {'ALL': {'C': 2, 'E': 0, '%': 0.0, 'S': -6}, 'SPE': {'C': 1, 'E': 0, '%': 50.0, 'S': 1}, 'ECO': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}, 'WHI': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}}, 'ISS': {'ALL': {'C': 30, 'E': 0, '%': 0.002, 'S': -5}, 'SPE': {'C': 29, 'E': 0, '%': 96.667, 'S': 6}, 'ECO': {'C': 20, 'E': 0, '%': 66.667, 'S': 1}, 'WHI': {'C':

       DISTRICT SECTION  all_punishments
17141     31901     EXP                2
17142     31901     DAE              602
17143     31901     OSS             3091
17144     31901     ISS             8413
17145    108902     EXP                2
{'DISTRICT_C': '228905', 'DISTRICT_N': 228905, 'OBJECTID': 31, 'NAME2': 'Apple Springs', 'DISTRICT': '228-905', 'OBJECTID_2': 1113, 'DISTNAME': 'APPLE SPRINGS ISD', 'REGION': 6, 2006: {'DPETALLC': 167, 'DPETALLP': 0.0, 'DPETBLAP': 16, 'DPETHISP': 2, 'DPETWHIP': 83, 'DPETECOP': 59.9, 'DPETSPEP': 20, 'EXP': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -1}}, 'DAE': {'ALL': {'C': 0, 'E': 0, '%': 0.0, 'S': -4}}, 'OSS': {'ALL': {'C': 2, 'E': 0, '%': 0.0, 'S': -6}, 'SPE': {'C': 1, 'E': 0, '%': 50.0, 'S': 1}, 'ECO': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}, 'WHI': {'C': 1, 'E': 0, '%': 50.0, 'S': -1}}, 'ISS': {'ALL': {'C': 30, 'E': 0, '%': 0.002, 'S': -5}, 'SPE': {'C': 29, 'E': 0, '%': 96.667, 'S': 6}, 'ECO': {'C': 20, 'E': 0, '%': 66.667, 'S': 1}, 'WHI': {'C':

In [10]:
pd.notnull(district.loc[228905])

DISTNAME    True
REGION      True
DPETALLC    True
DPETBLAP    True
DPETHISP    True
DPETWHIP    True
DPETINDP    True
DPETASIP    True
DPETPCIP    True
DPETTWOP    True
DPETECOP    True
DPETSPEP    True
DPETALLP    True
Name: 228905, dtype: bool

In [11]:
int(apple.loc[entry["id"],punishment,demo][year_col])

33

In [12]:
float(apple.loc[entry["id"],punishment,demo]["Percentage"])

1.275

In [13]:
apple.loc[entry["id"],punishment,demo]["Scale"]

2

In [14]:
with open('../geojson/districts_with_data.geojson', 'w') as fp:
    json.dump(district_map, fp, default=int)
    fp.close()

In [15]:
import geojson

fc = geojson.FeatureCollection(district_map)

In [16]:
type(fc)

geojson.feature.FeatureCollection

In [17]:
help(geojson.load)

Help on function load in module geojson.codec:

load(fp, cls=<class 'json.decoder.JSONDecoder'>, parse_constant=<function _enforce_strict_numbers at 0x11b2d7840>, object_hook=<bound method GeoJSON.to_instance of <class 'geojson.base.GeoJSON'>>, **kwargs)



In [18]:
with open("../geojson/districts_with_data.geojson") as geo_data:
    fc = geojson.load(geo_data)
    geo_data.close()

In [19]:
district_map["features"][3]

{'geometry': {'coordinates': [[[-100.27515251768764, 34.7111389239958],
    [-100.28657751973152, 34.71077492247225],
    [-100.31184752675881, 34.71081492105591],
    [-100.31216652699148, 34.703736920181406],
    [-100.31243352377226, 34.66611091322487],
    [-100.33106252812291, 34.66589691201054],
    [-100.3312295268639, 34.64954390931071],
    [-100.34276552863754, 34.649149909104864],
    [-100.34966653147917, 34.64928690860006],
    [-100.3499905290419, 34.62083190423951],
    [-100.37889753639715, 34.620883902110876],
    [-100.40052054204337, 34.621032901222385],
    [-100.41585854499239, 34.6209689004178],
    [-100.41592154324587, 34.589524896120565],
    [-100.41579454076648, 34.558511890082706],
    [-100.41595853781935, 34.51835488290953],
    [-100.41596053666917, 34.50006488057768],
    [-100.41684753212495, 34.42457586695365],
    [-100.42922853412152, 34.42467186533117],
    [-100.42928953339748, 34.41542686458377],
    [-100.4300625317414, 34.391619859961324],
    [

In [26]:
# Testing to see if the file we produced is valid GeoJSON

fc.is_valid

True

In [20]:
district.loc[67908]

DISTNAME    RISING STAR ISD
REGION                   14
DPETALLC                160
DPETBLAP                  0
DPETHISP               19.4
DPETWHIP               79.4
DPETINDP                  0
DPETASIP                  0
DPETPCIP                  0
DPETTWOP                1.3
DPETECOP               70.6
DPETSPEP                5.6
DPETALLP                  0
Name: 67908, dtype: object

In [21]:
len(fail)

4

In [22]:
len(noScale)

779

In [30]:
# df[df['A'] > 0]

q = district_map["features"][900]["properties"]["DISTRICT_N"]

district.loc[q]

# district[district["DISTRICT"] == 167903]

DISTNAME    RANDOLPH FIELD ISD
REGION                      20
DPETALLC                  1357
DPETBLAP                  17.5
DPETHISP                  21.9
DPETWHIP                  45.3
DPETINDP                   0.3
DPETASIP                   2.9
DPETPCIP                   1.1
DPETTWOP                  11.1
DPETECOP                   8.8
DPETSPEP                   7.1
DPETALLP                  0.03
Name: 15906, dtype: object